# Convolutional Neural Networks

<img src="mnist_sample.png", height=200, width=200>

## Task 1: Load the Data

Load the entire MNIST set from Tensorflow's dataset.

In [1]:
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# set batchSz batch gradient descent
batchSz = 50

# set hiddenSz for the hidden layer
hiddenSz = 1000 

# set learning rate 
l = 0.0004

/Users/MatiasBerretta/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
print(mnist.train.images.shape)

(55000, 784)


In [3]:
print(mnist.train.labels.shape)

(55000, 10)


# Placeholders:

In [4]:
# Images and their corresponding labels
img = tf.placeholder(tf.float32, [None,28,28,1])
ans = tf.placeholder(tf.float32, [None, 10])

## Architecture
* First convolution layer with 5x5 filter with 1 in channel and 32(k=32) in depth and SAME padding
* ReLU
* Maxpooling layer with 2x2 filter and SAME padding
* Second Convolution layer with 5x5 filter 32 in channel and 64 in depth(k=64)
* ReLU
* Maxpooling layer with 2x2 filter and SAME padding
* Feed-forward layer from 3136 (you should have 3136 parameters at this point) to 1000
* ReLU
* Feed-forward layer from 1000 to 10
* Softmax/Sigmoid


In [5]:

# First Convolution layer with 5x5 filters, 1 in channel, and 32 depth
# 50*28*28*1
conv1 = tf.layers.conv2d(inputs=img,filters=32,kernel_size=5,strides = 1,padding='same',activation=tf.nn.relu) # -> 32*28*28*1
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)  # -> 50*14*14*32

# Second convolution layer with 5x5 filters, 32 in channel, and 64 in depth
# 50*14*14*32
conv2 = tf.layers.conv2d(inputs=pool1,filters=64,kernel_size= 5,strides=1,padding='same',activation=tf.nn.relu) # 50*14*14*64
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)   # -> 50*7*7*64

pool2_flat = tf.reshape(pool2, [-1,7 * 7 * 64])

# initialize W and bw for the first hidden layer: W should be (64*7*7 by hiddensize), bw should be (hiddensize by 1)
wh1 = tf.Variable(tf.random_normal([64*7*7,hiddenSz],stddev = 0.1))
bh1 = tf.Variable(tf.random_normal([1,hiddenSz]))

feedf1 = tf.nn.relu(tf.matmul(pool2_flat,wh1)+bh1)

# initialize V and bv for the final layer: V should be (hiddensize by 10), and bv should be (10 by 1)
v = tf.Variable(tf.random_normal([hiddenSz,10],stddev = 0.1))
bv = tf.Variable(tf.random_normal([1,10],stddev = 0.1))

before_prbs = tf.matmul(feedf1,v)+bv
prbs = tf.nn.softmax(before_prbs)


## Forward and Backward Pass
Write the forward and backward pass in tensorflow
* Use cross entropy loss
* Use AdamOptimizer

In [ ]:
# Forward passs -- compute the cross entropy and output probabilities
xEnt = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=ans, logits=before_prbs))

# Backward pass
# Note: you don't need to compute the gradient 
# Simply use the tf.train.AdamOptimizer() function
train = tf.train.AdamOptimizer(0.0004).minimize(xEnt)

# Compute Accuracy
numCorrect = tf.equal(tf.argmax(prbs,1),tf.argmax(ans,1))
accuracy = tf.reduce_mean(tf.cast(numCorrect, tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

## Training the Network and Testing our Results
Train on 2000 batches

In [ ]:
start = time.time()

# Training on 2000 batches
for i in range(2000):
    imgs, anss = mnist.train.next_batch(batchSz)
    sess.run(train, feed_dict={img: imgs.reshape([batchSz,28,28,1]), ans: anss})


trainAcc=0
# Training accuracy
for i in range(len(mnist.train.images)):
    imgs, anss= mnist.train.next_batch(1)
    trainAcc += sess.run(accuracy, feed_dict={img: imgs.reshape([1,28,28,1]), ans: anss})
print ("Train Accuracy: %r" % (trainAcc/len(mnist.train.images)))

sumAcc=0
# Testing each image
for i in range(len(mnist.test.images)):
    imgs, anss= mnist.test.next_batch(1)
    sumAcc += sess.run(accuracy, feed_dict={img: imgs.reshape([1,28,28,1]), ans: anss})
print ("Test Accuracy: %r" % (sumAcc/len(mnist.test.images)))

end = time.time()
print("Runtime: %g" % (end - start))